In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113


In [3]:
!pip install -U openmim
!mim install mmcv-full

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 235 kB 62.2 MB/s 
     |████████████████████████████████| 51 kB 6.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.6 MB 1.2 MB/s 
     |████████████████████████████████| 190 kB 36.5 MB/s 


In [4]:
!rm -rf mmsegmentation #rm:remove rf:force directory
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 10458, done.
remote: Counting objects: 100% (333/333), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 10458 (delta 149), reused 273 (delta 127), pack-reused 10125
Receiving objects: 100% (10458/10458), 14.59 MiB | 12.70 MiB/s, done.
Resolving deltas: 100% (7579/7579), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
     |████████████████████████████████| 578 kB 37.8 MB/s 
  Running setup.py develop for mmsegmentation


In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.12.0+cu113 True
0.27.0


In [7]:
#To verify whether MMSegmentation is installed correctly, we provide some sample codes to run an inference demo.

#Step 1. We need to download config and checkpoint files.

!mim download mmsegmentation --config pspnet_r50-d8_512x1024_40k_cityscapes --dest .

processing pspnet_r50-d8_512x1024_40k_cityscapes...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 MiB 34.3 MB/s eta 0:00:00
Successfully downloaded pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth to /content/mmsegmentation
Successfully dumped pspnet_r50-d8_512x1024_40k_cityscapes.py to /content/mmsegmentation


In [11]:
!mkdir samplevid
!wget https://download.samplelib.com/mp4/sample-5s.mp4 -P samplevid

--2022-08-10 11:09:20--  https://download.samplelib.com/mp4/sample-5s.mp4
Resolving download.samplelib.com (download.samplelib.com)... 2.56.91.35
Connecting to download.samplelib.com (download.samplelib.com)|2.56.91.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2848208 (2.7M) [video/mp4]
Saving to: ‘samplevid/sample-5s.mp4’

sample-5s.mp4       100%[===================>]   2.72M  1.58MB/s    in 1.7s    

2022-08-10 11:09:22 (1.58 MB/s) - ‘samplevid/sample-5s.mp4’ saved [2848208/2848208]



In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
import mmcv

config_file = 'pspnet_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file = 'pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
img = '/content/mmsegmentation/demo/demo.png'  # or img = mmcv.imread(img), which will only load it once
result = inference_segmentor(model, img)
# visualize the results in a new window
show_result_pyplot(model, img, result, get_palette('cityscapes'))
#model.show_result(img, result, show=True) # bu hata veriyor
# or save the visualization results to image files
# you can change the opacity of the painted segmentation map in (0, 1].
#model.show_result(img, result, out_file='result.jpg', opacity=0.5)
show_result_pyplot(model, img, result, get_palette('cityscapes'),out_file='result.jpg', opacity=0.1)

# test a video and show the results
# çok uzun sürüyor
'''
video = mmcv.VideoReader('/content/mmsegmentation/samplevid/sample-5s.mp4')
for frame in video:
   result = inference_segmentor(model, frame)
   #model.show_result(frame, result, wait_time=1)
   show_result_pyplot(model, frame, result, get_palette('cityscapes'))
'''
